<a href="https://colab.research.google.com/github/sharonmustri/oxfordshire_power/blob/main/oxford_power_full_sharon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Create network model for the Oxford 

In [ ]:
# install pandapower in colab
!pip install pandapower

     |████████████████████████████████| 5.8 MB 12.4 MB/s 
     |████████████████████████████████| 149 kB 38.8 MB/s 
     |████████████████████████████████| 3.6 MB 32.2 MB/s 
  Created wheel for pandapower: filename=pandapower-2.7.0-py3-none-any.whl size=5777359 sha256=fb96ccb0f1fc2d2f29ff6028808f339710d56fe4c13c0a7ef270b27337c37913
  Stored in directory: /root/.cache/pip/wheels/ff/e6/b6/c9218d5a45a02f4bdb220272f2bac4f59c5b73ed08a08125c3
Successfully built pandapower


In [ ]:
# imports
import pandas as pd
import pandapower as pp
from pandapower.plotting.plotly import simple_plotly

In [ ]:
# to do

# check trafo numbers
# check line specs
# add detail

In [ ]:
# key variables
ADMD_MW = 2e-3
pv_MWp = 3.4e-3
Q_MVAR = 24.2e-3/25

In [ ]:
# create network
net = pp.create_empty_network() 

In [ ]:
# layout characteristics
start_x = 0
start_y = 0

# distance between slack bus and 132/11 trafo
dist_slack_trafo11 = 0.1
# distance between 132/11 trafo and first house
dist_trafo11_hh = 0.04
# length of one side of household square plot
hh_side_km = 0.05
# distance between main road and first house
dist_main_hh = 0.05
# length of a full group of 25 households
len_hh_group = 0.225
# space between each group of 25 households
len_bw_group = 0.02

In [ ]:
# create bus dict
b = {}

In [ ]:
# one bus at 132kV for grid connection
x, y = start_x, start_y
b[0] = pp.create_bus(net, vn_kv = 132, name = f"Bus 0", geodata = (x,y))

# one bus at 11kV for first step down
x, y = x + dist_slack_trafo11, y
b[1] = pp.create_bus(net, vn_kv = 11, name = f"Bus 1", geodata = (x,y))

# create four buses at 415V for each transformer
for i in range(2,6):
  if i == 2:
    x, y = x + dist_trafo11_hh + len_hh_group/2, y
  else: 
    x, y = x + len_hh_group + len_bw_group, y
  b[i] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i}", geodata = (x,y))
  # add bus for each of the 3 bundles up and down of the trafo representing 25 houses
  x, y = x, start_y + dist_main_hh + hh_side_km*3
  b[i*10+1] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i}", geodata = (x,y))
  b[i*10+2] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i}", geodata = (x,-y))
  x, y = x, start_y + dist_main_hh + hh_side_km*6
  b[i*10+3] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i}", geodata = (x,y))
  b[i*10+4] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i}", geodata = (x,-y))
  x, y = x, start_y + dist_main_hh + hh_side_km*7
  b[i*10+5] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i}", geodata = (x,y))
  b[i*10+6] = pp.create_bus(net, vn_kv = 0.415, name = f"Bus {i}", geodata = (x,-y))
  # reset y for next trafo
  y = 0

In [ ]:
# create one grid connection point at b0
pp.create_ext_grid(net, bus=b[0], vm_pu=1, name = "Slack bus")

0

In [ ]:
# read trafo data
trf = pd.read_csv('/content/Assignment Data - Oxfordshire - Sheet1.csv')
trf = trf.T
trf.columns = trf.loc['Parameter', :].to_list()
trf = trf.drop(labels=['Parameter', 'Datatype', 'Explanation'])

In [ ]:
# create one trafo b0 to b1 - from 123 to 11
hv_bus = b[0]
lv_bus = b[1]
trf_row = '132/11'

sn_mva = float(trf.loc[trf_row, 'sn_mva*'])
vn_hv_kv = float(trf.loc[trf_row, 'vn_hv_kv*'])
vn_lv_kv = float(trf.loc[trf_row, 'vn_lv_kv*'])
vk_percent = float(trf.loc[trf_row, 'vk_percent*'])
vkr_percent = float(trf.loc[trf_row, 'vkr_percent*'])
pfe_kw = float(trf.loc[trf_row, 'pfe_kw*'])
i0_percent = float(trf.loc[trf_row, 'i0_percent*'])

pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

0

In [ ]:
# create four trafos - from 11 to .415
trf_row = '11/0.415'

sn_mva = float(trf.loc[trf_row, 'sn_mva*'])
vn_hv_kv = float(trf.loc[trf_row, 'vn_hv_kv*'])
vn_lv_kv = float(trf.loc[trf_row, 'vn_lv_kv*'])
vk_percent = float(trf.loc[trf_row, 'vk_percent*'])
vkr_percent = float(trf.loc[trf_row, 'vkr_percent*'])
pfe_kw = float(trf.loc[trf_row, 'pfe_kw*'])
i0_percent = float(trf.loc[trf_row, 'i0_percent*'])

for i in range(2,6):
  hv_bus = b[1]
  lv_bus = b[i]

  pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

In [ ]:
#### create line at 132kV from b0 to b1 (not sure if malcome said to ignore this?)

#### THESE parameters are wrong, just copied from below for illustration
r_ohm_per_km = 0.5     
x_ohm_per_km = 0.09
c_nf_per_km = 1e-9
max_i_ka = 0.15
length_km = dist_slack_trafo11

from_bus = b[0]
to_bus = b[1]
pp.create_line_from_parameters(net, from_bus, to_bus, length_km, r_ohm_per_km, x_ohm_per_km, c_nf_per_km, max_i_ka) 

0

In [ ]:
# create line at 11kV from b1 to b2 to b3 to b4 to b5

#### THESE parameters are wrong, just copied from below for illustration
r_ohm_per_km = 0.5     
x_ohm_per_km = 0.09
c_nf_per_km = 1e-9
max_i_ka = 0.15

for i in range(1,5):
  from_bus = b[i]
  to_bus = b[i+1]
  if i == 1:
    length_km = dist_trafo11_hh + len_hh_group/2
  else:
    length_km = len_hh_group + len_bw_group
  pp.create_line_from_parameters(net, from_bus, to_bus, length_km, r_ohm_per_km, x_ohm_per_km, c_nf_per_km, max_i_ka) 

In [ ]:
# create upward/downward lines at 415V from each trafo

#### need to check these numbers
r_ohm_per_km = 0.5     # from malcom's chart in 08_01 
x_ohm_per_km = 0.09
c_nf_per_km = 1e-9
max_i_ka = 0.15

for i in range(2,6):
  from_bus = b[i]   
  for j in range(1,4):
    length_km = 0.2
    # upward
    to_bus = b[i*10+j]
    pp.create_line_from_parameters(net, from_bus, to_bus, length_km, r_ohm_per_km, x_ohm_per_km, c_nf_per_km, max_i_ka) 
    # downward
    to_bus = b[i*10+j+3]
    pp.create_line_from_parameters(net, from_bus, to_bus, length_km, r_ohm_per_km, x_ohm_per_km, c_nf_per_km, max_i_ka)
    length_km = length_km - j*0.05

In [ ]:
# create loads
for i in range(2,6):
  for j in range(1,4):
    if j ==3:
      p_mw = ADMD_MW
      q_mvar = Q_MVAR
    else: 
      p_mw = ADMD_MW * 12
      q_mvar = Q_MVAR * 12
    # upward
    pp.create_load(net, b[i*10+j], p_mw=p_mw, q_mvar=q_mvar)
    # downward
    pp.create_load(net, b[i*10+j+3], p_mw=p_mw, q_mvar=q_mvar)

In [ ]:
# create pv generation at buses 6 to 13
for i in range(2,6):
  for j in range(1,4):
    if j ==3:
      p_mw = (-1) * pv_MWp
      q_mvar = 0
    else: 
      p_mw = (-1) * pv_MWp * 12
      q_mvar = 0
    # upward
    pp.create_load(net, b[i*10+j], p_mw=p_mw, q_mvar=q_mvar)
    # downward
    pp.create_load(net, b[i*10+j+3], p_mw=p_mw, q_mvar=q_mvar)

In [ ]:
pp.runpp(net, algorithm='gs');

In [ ]:
simple_plotly(net)

In [ ]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,-0.263442,0.245504,0.263443,-0.245504,3.721131e-07,6.698035e-08,0.001575,0.018900,0.018900,1.000000,0.000000,1.000001,0.000048,12.600327
1,-0.065234,0.116303,0.065245,-0.116301,1.120540e-05,2.016972e-06,0.006999,0.185514,0.185514,1.000001,0.000048,1.000029,0.004671,123.676306
2,0.003101,0.064769,-0.000110,-0.064231,2.990480e-03,5.382864e-04,0.090207,0.090207,0.090207,1.000029,0.004671,0.990584,2.647201,60.138212
3,0.014980,0.028113,-0.014245,-0.027981,7.355723e-04,1.324030e-04,0.044739,0.044739,0.044739,0.990584,2.647201,0.976365,3.718244,29.825844
4,0.009682,0.009250,-0.009548,-0.009226,1.337810e-04,2.408059e-05,0.019080,0.019080,0.019080,0.976365,3.718244,0.968114,4.041908,12.719714
5,-0.016562,0.011659,0.016800,-0.011616,2.381767e-04,4.287181e-05,0.028177,0.028177,0.028177,1.000029,0.004671,1.008462,0.487620,18.784419
6,-0.016562,0.011659,0.016800,-0.011616,2.381767e-04,4.287181e-05,0.028177,0.028177,0.028177,1.000029,0.004671,1.008462,0.487620,18.784419
7,-0.016562,0.011659,0.016800,-0.011616,2.381767e-04,4.287181e-05,0.028177,0.028177,0.028177,1.000029,0.004671,1.008462,0.487620,18.784419
8,-0.016562,0.011659,0.016800,-0.011616,2.381767e-04,4.287181e-05,0.028177,0.028177,0.028177,1.000029,0.004671,1.008462,0.487620,18.784419
9,-0.001398,0.000968,0.001400,-0.000968,1.679631e-06,3.023335e-07,0.002366,0.002366,0.002366,1.000029,0.004671,1.000739,0.045227,1.577448


In [ ]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,0.260952,-0.257734
1,1.000001,0.000048,0.000000,0.000000
2,1.000029,0.004671,0.000000,0.000000
3,1.008462,0.487620,-0.016800,0.011616
4,1.008462,0.487620,-0.016800,0.011616
5,1.000739,0.045227,-0.001400,0.000968
6,1.008462,0.487620,-0.016800,0.011616
7,1.008462,0.487620,-0.016800,0.011616
8,1.000739,0.045227,-0.001400,0.000968
9,0.990584,2.647201,0.000000,0.000000
